In [32]:
import sys, os
from gears import PertData
import scanpy as sc
import anndata as ad
import pickle

In [33]:
pathway = 'IFNG'
cell_type = 'BXPC3'

In [42]:
data_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/satija/datasets/r2"
adata_train = sc.read_h5ad(os.path.join(data_path, "adata_train_" + pathway + "_" + cell_type + ".h5ad"))
adata_train = adata_train[adata_train.obs['cell_type'] == cell_type, :]
adata_ood = sc.read_h5ad(os.path.join(data_path, "adata_ood_" + pathway + "_" + cell_type + ".h5ad"))
adata_ood = adata_ood[adata_ood.obs['cell_type'] == cell_type, :]
adata = ad.concat([adata_train, adata_ood], join='outer')
adata

AnnData object with n_obs × n_vars = 61522 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'cell_type', 'percent.mito', 'sample_ID', 'Batch_info', 'bc1_well', 'bc2_well', 'bc3_well', 'guide', 'gene', 'mixscale_score', 'condition', 'control', 'ood', 'is_ood', 'split'
    obsm: 'X_pca', 'cell_line_emb', 'cond_emb', 'gene_emb'
    layers: 'X_log1p', 'centered_X'

In [43]:
ood_genes = adata_ood.obs[adata_ood.obs['gene'] != 'NT']['gene'].unique().tolist()
ood_genes

['TRAFD1', 'EHF', 'ATF5', 'MAFB', 'IFI16', 'IRF7', 'STAT1']

In [53]:
train_genes = adata_train.obs[adata_train.obs['gene'] != 'NT']['gene'].unique().tolist()

In [54]:
adata.obs['condition'] = adata.obs.apply(lambda x: "+".join([x.gene, 'ctrl']), axis=1)
adata.obs.loc[adata.obs['gene'] == 'NT', 'condition'] = 'ctrl'
adata.obs['condition'].unique()

array(['IRF9+ctrl', 'PRDM1+ctrl', 'BATF2+ctrl', 'FOXN3+ctrl',
       'PTPN11+ctrl', 'PARP12+ctrl', 'IRF1+ctrl', 'IFNGR2+ctrl',
       'RARRES3+ctrl', 'RUNX1+ctrl', 'IFNGR1+ctrl', 'ATF3+ctrl',
       'CEBPB+ctrl', 'ETV7+ctrl', 'ctrl', 'SP100+ctrl', 'TBX21+ctrl',
       'IRF2+ctrl', 'MYC+ctrl', 'HLA-DQB1+ctrl', 'JAK1+ctrl', 'JAK2+ctrl',
       'JUN+ctrl', 'RFX5+ctrl', 'STAT3+ctrl', 'MAFF+ctrl', 'FMNL2+ctrl',
       'PLEK+ctrl', 'GUK1+ctrl', 'SRC+ctrl', 'RNF14+ctrl', 'CEBPE+ctrl',
       'TRAFD1+ctrl', 'EHF+ctrl', 'ATF5+ctrl', 'MAFB+ctrl', 'IFI16+ctrl',
       'IRF7+ctrl', 'STAT1+ctrl'], dtype=object)

In [55]:
adata

AnnData object with n_obs × n_vars = 61522 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'cell_type', 'percent.mito', 'sample_ID', 'Batch_info', 'bc1_well', 'bc2_well', 'bc3_well', 'guide', 'gene', 'mixscale_score', 'condition', 'control', 'ood', 'is_ood', 'split'
    var: 'gene_name'
    obsm: 'X_pca', 'cell_line_emb', 'cond_emb', 'gene_emb'
    layers: 'X_log1p', 'centered_X'

In [62]:
adata.obs.query('condition == "RARRES3+ctrl"')

,orig.ident,nCount_RNA,nFeature_RNA,sample,cell_type,percent.mito,sample_ID,Batch_info,bc1_well,bc2_well,...,guide,gene,mixscale_score,condition,control,ood,is_ood,split,dose_val,condition_name
28_93_82_1_1_1_1_1_1_1_1_1,28,8609.0,3870,BXCP3_IFNG,BXPC3,5.273550935067952,sample_1,Rep1,nan,nan,...,RARRES3g1,RARRES3,1.2247992493382465,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
28_48_62_1_1_1_1_1_1_1_1_1,28,4662.0,2690,BXCP3_IFNG,BXPC3,3.9468039468039464,sample_1,Rep1,nan,nan,...,RARRES3g1,RARRES3,0.7364614611075123,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
26_27_25_1_1_1_1_1_1_1_1_1,26,4663.0,2686,BXCP3_IFNG,BXPC3,4.717992708556723,sample_1,Rep1,nan,nan,...,RARRES3g1,RARRES3,1.9977108311345417,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
28_91_62_1_1_1_1_1_1_1_1_1,28,2728.0,1848,BXCP3_IFNG,BXPC3,3.225806451612903,sample_1,Rep1,nan,nan,...,RARRES3g1,RARRES3,2.968267133668633,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
27_12_90_1_1_1_1_1_1_1_1_1,27,2355.0,1632,BXCP3_IFNG,BXPC3,2.5053078556263273,sample_1,Rep1,nan,nan,...,RARRES3g1,RARRES3,1.230744635345631,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21_72_45_2_2,21,2100.0,1435,BXPC3_IFNG,BXPC3,11.047619047619047,sample_16,Rep2,B9,F12,...,RARRES3g1,RARRES3,1.0152437782988055,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
21_71_51_2_2,21,1866.0,1259,BXPC3_IFNG,BXPC3,5.359056806002144,sample_16,Rep2,B9,F11,...,RARRES3g1,RARRES3,1.2814108219512041,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
23_18_27_2_2,23,1445.0,1084,BXPC3_IFNG,BXPC3,2.006920415224913,sample_16,Rep2,B11,B6,...,RARRES3g1,RARRES3,0.7700884061860296,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1
21_03_96_2_2,21,1295.0,1030,BXPC3_IFNG,BXPC3,3.2432432432432434,sample_16,Rep2,B9,A3,...,RARRES3g1,RARRES3,1.5431941835285357,RARRES3+ctrl,0,False,False,train,1+1,BXPC3_RARRES3+ctrl_1+1


In [56]:
adata.var['gene_name'] = adata.var.index

In [57]:
pert_data = PertData('./data/satija') 
pert_data.new_data_process(dataset_name = 'satija_' + pathway.lower() + "_" + cell_type.lower(), adata = adata)

Downloading...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.46M/9.46M [00:00<00:00, 9.97MiB/s]
Downloading...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 559k/559k [00:00<00:00, 1.07MiB/s]
Creating pyg object for each cell in the data...
Creating dataset file...
 21%|███████████████████████████████████▎                                                                                                                                        | 8/39 [01:04<04:53,  9.45s/it]

RARRES3+ctrl


 49%|███████████████████████████████████████████████████████████████████████████████████▎                                                                                       | 19/39 [02:46<02:27,  7.37s/it]

HLA-DQB1+ctrl


 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                         | 26/39 [03:11<00:38,  2.95s/it]

...

 74%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                           | 29/39 [03:21<00:33,  3.30s/it]

SRC+ctrl


 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊             | 36/39 [03:53<00:16,  5.55s/it]

IFI16+ctrl


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [04:00<00:00,  6.17s/it]
Done!
Saving new dataset pyg object at ./data/satija/satija_ifng_bxpc3/data_pyg/cell_graphs.pkl
Done!


In [61]:
custom_split_dict = {'train': [x  + '+ctrl' for x in train_genes], 'val': ['RUNX1+ctrl'], 'test': [x  + '+ctrl' for x in ood_genes]}
custom_split_path = './data/satija/satija_ifng_bxpc3/custom_split.pkl'
with open(custom_split_path, 'wb') as file:
    pickle.dump(custom_split_dict, file)

In [21]:
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 512, test_batch_size = 512) # prepare data loader

Creating new splits....
Saving new splits at ./data/satija/satija_ifng_hvg_2_bxpc3/splits/satija_ifng_hvg_2_bxpc3_single_1_0.75.pkl
Done!
Creating dataloaders....
Done!


In [29]:
pert_data

TSPAN6            TSPAN6
TNMD                TNMD
DPM1                DPM1
SCYL3              SCYL3
C1orf112        C1orf112
                 ...    
AC068620.2    AC068620.2
AC006023.1    AC006023.1
AL356121.2    AL356121.2
TAF9BP2          TAF9BP2
AC016065.2    AC016065.2
Name: gene_name, Length: 33525, dtype: object